In [ ]:
conda install tensorflow

In [ ]:
conda install keras

In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load dataset and split them into train and test set

In [2]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/'
#####################   Modify above path   #########################
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)
X_dis=X
y_dis=y

# MLPClassifier

### Use GridsearchCV to determine Parameters

In [3]:
# from sklearn.model_selection import GridSearchCV
# MLP_parm_gs = {
#     'hidden_layer_sizes': [(1000,), (1000,1000), (2000,), (2000,1000), (3000,), (1000,2000), (1000,1000,1000)],
#     'learning_rate':['adaptive', 'constant'],
#     'solver':['lbfgs','sgd','adam'],
#     'early_stopping':[True] , 
#     'validation_fraction':[0.2],
                        
# }
# MLP_model_gs = MLPClassifier()
# MLP_gsearch = GridSearchCV(estimator = MLP_model_gs, 
#                         param_grid = MLP_parm_gs, 
#                         scoring ='accuracy',
#                         cv = 3,
#                         n_jobs = 2,
#                         verbose=3,
#                         return_train_score=True
#                        )
# MLP_gsearch.fit(train_x_dis,train_y_dis)

Fitting 3 folds for each of 42 candidates, totalling 126 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 41.3min
[Parallel(n_jobs=2)]: Done 126 out of 126 | elapsed: 231.8min finished
E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_sta...
                                     warm_start=False),
             iid='warn', n_jobs=2,
             param_grid={'early_stopping': [True],
                         'hidden_layer_sizes': [(1000,), (1000, 1000), (2000,),
                                                (2000, 1000), (3000,),
                           

In [4]:
# from sklearn.metrics import accuracy_score 
# print("Training dataset Accuracy")
# train_preds = MLP_gsearch.predict(train_x_dis)
# train_accuracy = accuracy_score(train_y_dis, train_preds) 
# print(train_accuracy)

Training dataset Accuracy
1.0


In [5]:
# print("Testing dataset Accuracy")
# test_preds = MLP_gsearch.predict(test_x_dis)
# test_accuracy = accuracy_score(test_y_dis, test_preds) 
# print(test_accuracy)

Testing dataset Accuracy
0.49


In [6]:
# MLP_gsearch.best_params_

{'early_stopping': True,
 'hidden_layer_sizes': (3000,),
 'learning_rate': 'adaptive',
 'solver': 'lbfgs',
 'validation_fraction': 0.2}

### Use K-Fold to estimate the accuracy (49.44%)

In [9]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(3000,),
                              learning_rate='adaptive',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [10]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(MLP_model_final, X_dis,y_dis, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 49.44% (4.31%)


### Fit the final model and save it

In [11]:
t0 = time.time()
MLP_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

done in 171.824s


In [12]:
# joblib.dump(MLP_model_final,'MLP_model_final.m')#please change the path

['MLP_model_final.m']

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/MLP_model_final.m')